In [8]:
from transformers import BertTokenizer
from transformers import BertForQuestionAnswering

import torch

In [49]:
# Initialize tokenizer for corpus of bert-large-uncased
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

# Initialize model BertForQuestionAnswering for bert-large-uncased
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [53]:
def answer_question(question, answer_text):
    '''
    Lấy input là chuỗi string của câu question và answer_text chứa nội dung câu trả lời của câu question.
    Xác định từ trong answer_text là câu trả lời và in ra.
    '''
    # ======== Tokenize ========
    # Áp dụng tokenizer cho cặp câu <question, answer_text>. input_ids là concatenate indice của cả 2 câu sau khi đã thêm các token CLS và SEP như mô tả trong tác vụ Question and Answering.
    input_ids = tokenizer.encode(question, answer_text)

    # ======== Set Segment IDs ========
    # Xác định vị trí đầu tiên chứa token [SEP] trong câu.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # Tạo segment index đánh dấu các vị trí từ thuộc question (giá trị 0) và answer_text (giá trị 1)
    num_seg_a = sep_index + 1
    num_seg_b = len(input_ids) - num_seg_a
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # Kiểm tra độ dài segment_ids phải bằng input_ids
    assert len(segment_ids) == len(input_ids)
    
    input_ids = torch.tensor([input_ids])
    segment_ids = torch.tensor([segment_ids])

    # ======== Evaluate ========
    # Dự báo phân phối xác suất của vị trí của từ start và từ end trong chuỗi concatenate <question, answer_text> mà chứa kết quả cho câu trả lời.
    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=segment_ids)
        start_scores, end_scores = outputs.start_logits, outputs.end_logits

    # ======== Reconstruct Answer ========
    # Tìm ra vị trí start, end với score là cao nhất
    answer_start = torch.argmax(start_scores, dim=1).item()
    answer_end = torch.argmax(end_scores, dim=1).item()

    # Chuyển ngược từ input_ids sang list tokens
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    print(tokens)

    # Token đầu tiên của câu trả lời
    answer = tokens[answer_start]

    # Lựa chọn các thành phần còn lại của câu trả lời và join chúng với whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        
        # Nếu token là một subword token (có dấu ## ở đầu) thì combine vào answer bằng token gốc (loại bỏ dấu ##).
        if tokens[i].startswith('##'):
            answer += tokens[i][2:]
        
        # Nếu trái lại thì combine trực tiếp vào answer.
        else:
            answer += ' ' + tokens[i]
            
    print('Question: "' + question + '"')
    print('Answer: "' + answer + '"')

In [54]:
question = "what is my dog name?"
paragraph = "I have a dog. It's name is Ricky. I get it at my 15th birthday, when it was a puppy."

answer_question(question, paragraph)

['[CLS]', 'what', 'is', 'my', 'dog', 'name', '?', '[SEP]', 'i', 'have', 'a', 'dog', '.', 'it', "'", 's', 'name', 'is', 'ricky', '.', 'i', 'get', 'it', 'at', 'my', '15th', 'birthday', ',', 'when', 'it', 'was', 'a', 'puppy', '.', '[SEP]']
Question: "what is my dog name?"
Answer: "ricky"


In [55]:
question = "What is the capital of France?"
answer_text = "The capital of France is Paris."
answer_question(question, answer_text)

['[CLS]', 'what', 'is', 'the', 'capital', 'of', 'france', '?', '[SEP]', 'the', 'capital', 'of', 'france', 'is', 'paris', '.', '[SEP]']
Question: "What is the capital of France?"
Answer: "paris"
